In [1]:
#short script to control actinic light source (Schott KL2500) via serial connection

#import statements
import serial
import serial.tools.list_ports
import time
import configparser
import os
import cv2
from ximea import xiapi
import RLD_manager

#functions
def set_actinic_light_int(serial_connection, intensity_permille):
    """set actinic light intensity in permille (0-1000), returns response from device"""
    # intensity_permille: int from 0 to 1000 convert to hexadecimal
    intensity = max(0, min(1000, intensity_permille))
    
    hex_intensity = f"{intensity:04X}"
    command = f"0BR{hex_intensity};\n"
    serial_connection.write(command.encode())
    response = serial_connection.readline().decode().strip()
    return response 


def list_serial_ports():
    ports = serial.tools.list_ports.comports()
    return [(port.device, port.description) for port in ports]

def select_serial_port():
    ports = list_serial_ports()
    if not ports:
        print("No serial ports available.")
        return None

    print("Available serial ports:")
    for i, (device, description) in enumerate(ports):
        print(f"{i + 1}: {device} - {description}")
    while True:
        try:
            selection = input("Select a serial port by number or enter \"q\" to quit: ")
            if selection == "q":
                return None
            selection = int(selection) - 1
            if 0 <= selection < len(ports):
                return ports[selection][0]  # Return the device name
            else:
                print("Invalid selection. Please try again.")
        except ValueError:
            #if selection.lower() == 'q':
            #    return None
            print("Invalid input. Please enter a number.")

def measure(rld, save_folder):
    rld.init_rld_controller()
    rld.acquire_images()
    rld.calculate_average_lifetime()
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    save_measurement(rld, file_path = save_folder)

def save_measurement(rld, file_path = None):
    if rld and rld.image_dict and len(rld.image_dict.get("window1", [])) > 0 and len(rld.image_dict.get("window2", [])) > 0 and len(rld.image_dict.get("dark", [])) > 0:
        for i, img in enumerate(rld.image_dict.get("window1", [])):
            cv2.imwrite(os.path.join(file_path, f"window1_{i:03d}.tif"), img) 
        for i, img in enumerate(rld.image_dict.get("window2", [])):
            cv2.imwrite(os.path.join(file_path, f"window2_{i:03d}.tif"), img) 
        for i, img in enumerate(rld.image_dict.get("dark", [])):
            cv2.imwrite(os.path.join(file_path, f"dark_{i:03d}.tif"), img)
        if rld.average_lifetime is not None:
            cv2.imwrite(os.path.join(file_path, "lifetime_image.tif"), rld.average_lifetime)

    config = configparser.ConfigParser()
    config['ImagingParameters'] = {
        'exposure_time_us': str(rld.params.exposure_time_us),
        'delay_window1_us': str(rld.params.delay_window1_us),
        'delay_window2_us': str(rld.params.delay_window2_us),
        'end_delay_us': str(rld.params.end_delay_us),
        'pulse_width_us': str(rld.params.pulse_width_us),
        'light_intensity': str(rld.params.light_intensity),
        'sets_to_acquire': str(rld.params.sets_to_acquire),
        'exposures_per_frame': str(rld.params.exposures_per_frame),
    }
    with open(os.path.join(file_path, 'settings.conf'), 'w') as configfile:
            config.write(configfile)


    
    if rld.start_time_ns and rld.end_time_ns:    # this is not guaranteed to be available if images were loaded from a folder
        #write timestamps to a text file
        with open(os.path.join(file_path, 'timestamps.txt'), 'w') as timestamps_file:
            #start time
            timestamps_file.write(f"Start timestamp: {rld.start_time_ns}\n")
            timestamps_file.write(f"End timestamp: {rld.end_time_ns}\n")
            timestamps_file.write(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S', rld.start_time_localtime)}:{rld.start_time_localtime_ms:.3f}\n")
            timestamps_file.write(f"End time: {time.strftime('%Y-%m-%d %H:%M:%S',rld.end_time_localtime)}:{rld.end_time_localtime_ms:.3f}\n")
            timestamps_file.write("Image acquisition timestamps (window, index, timestamp):\n")
            for key, ts_list in rld.image_start_time_dict.items():
                for index, timestamp in enumerate(ts_list):
                    timestamps_file.write(f"{key}, {index}, {timestamp}\n")




In [ ]:

#connect to actinic light source
serial_port_light = select_serial_port()
serial_connection_light = serial.Serial(serial_port_light, baudrate=9600, timeout=1)

set_actinic_light_int(serial_connection_light, 0) #set light intensity to 0 permille on start

In [ ]:

#connect to rld controller
serial_port_controller = select_serial_port()
serial_connection_controller = serial.Serial(serial_port_controller, baudrate=115200, timeout=1)

#open camera
camera = xiapi.Camera()
rld = RLD_manager.RLD()
camera.open_device()
rld.attach_hardware(camera, serial_connection_controller)


In [ ]:
save_folder = r"./test" #folder the images are saved to
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# set RLD parameters
rld.params.exposures_per_frame = 20 # mainly use standard parameters
rld.params.end_delay_us = 35
rld.init_camera()

# run measurement sequence e.g. 
set_actinic_light_int(serial_connection_light, 0)  # shut down actinic light 
for i in range (9):
    folder = os.path.join(save_folder, f"light_{i}_min")
    measure(rld, save_folder = folder)
    time.sleep(60) #wait one minute between measurements. exact timing found in timestamps.txt

folder = os.path.join(save_folder, f"light_{i+1}_min") #measure once more in light for (almost) gapless transition between light and dark 
measure(rld, save_folder = folder)

#switch on actinic light
set_actinic_light_int(serial_connection_light, 60)  # set actinic light to 60 permille
for j in range (10):
    folder = os.path.join(save_folder, f"dark_{i+1+j}min")
    measure(rld, save_folder = folder)
    time.sleep(60)
    

In [ ]:
#disconnect serial connection of actinic light source
serial_connection_light.close()

In [ ]:
#disconnect serial connection of rld controller and close camera
serial_connection_controller.close()
camera.close_device()